### Take  a  real-world  dataset  (e.g.,  Titanic,  House  Prices,  or  a  custom  dataset).  Use Pandas  to  handle  missing  values,  duplicates,  and  inconsistent  formats.  Implement Min-Max  scaling,  Z-score  normalization,  and  log  transformation  on  a  dataset. Compare how different scaling techniques affect model performance

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
# Load Titanic dataset (you can also use 'train.csv' from Kaggle)
df = pd.read_csv('titanic.csv')
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Basic information about the dataset
df.info()
df.describe(include='all').T

# Check for missing values and duplicates
print("\nMissing values per column:\n", df.isnull().sum())
print("\nNumber of duplicate rows:", df.duplicated().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB

Missing values per column:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embark

In [5]:
# Remove duplicates
df.drop_duplicates(inplace=True)

# Handle missing values
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
df['Cabin'].fillna('Unknown', inplace=True)

# Handle inconsistent formats (e.g., strings with spaces)
df['Sex'] = df['Sex'].str.strip().str.lower()
df['Embarked'] = df['Embarked'].str.strip().str.upper()

print("✅ Data cleaned successfully.")


✅ Data cleaned successfully.


C:\Users\omvis\AppData\Local\Temp\ipykernel_42088\1737811592.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\omvis\AppData\Local\Temp\ipykernel_42088\1737811592.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [6]:
# Select numeric and categorical features
df = df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Sex']]

# Encode categorical variable 'Sex' as binary (0/1)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# Split into features (X) and target (y)
X = df.drop('Survived', axis=1)
y = df['Survived']

X.head()


,Pclass,Age,SibSp,Parch,Fare,Sex
0,3,22.0,1,0,7.2500,0
1,1,38.0,1,0,71.2833,1
2,3,26.0,0,0,7.9250,1
3,1,35.0,1,0,53.1000,1
4,3,35.0,0,0,8.0500,0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [8]:
# 1️⃣ Min-Max Scaling
minmax_scaler = MinMaxScaler()
X_train_minmax = minmax_scaler.fit_transform(X_train)
X_test_minmax = minmax_scaler.transform(X_test)

# 2️⃣ Z-Score Normalization
zscore_scaler = StandardScaler()
X_train_zscore = zscore_scaler.fit_transform(X_train)
X_test_zscore = zscore_scaler.transform(X_test)

# 3️⃣ Log Transformation (only for positive numeric features)
X_train_log = np.log1p(X_train)  # log1p = log(1+x)
X_test_log = np.log1p(X_test)

print("✅ Scaling transformations completed.")


✅ Scaling transformations completed.


In [9]:
def train_and_evaluate(X_train, X_test, y_train, y_test, label):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    print(f"{label} Accuracy: {acc:.4f}")
    return acc

acc_minmax = train_and_evaluate(X_train_minmax, X_test_minmax, y_train, y_test, "Min-Max Scaling")
acc_zscore = train_and_evaluate(X_train_zscore, X_test_zscore, y_train, y_test, "Z-Score Normalization")
acc_log = train_and_evaluate(X_train_log, X_test_log, y_train, y_test, "Log Transformation")


Min-Max Scaling Accuracy: 0.7877
Z-Score Normalization Accuracy: 0.7989
Log Transformation Accuracy: 0.7989
